In [1]:
import requests
import json

import pandas as pd

from datetime import datetime,timedelta

In [2]:
token='c9439088efedf3d47e3efcfa4efe14d1'

In [3]:
url='http://apiadvisor.climatempo.com.br/api-manager/user-token/{}/locales'.format(token)
response=requests.get(url)
response.json() # checking which cities are registered in our app

{'max_allowed': 1, 'locales': [73779]}

In [4]:
city='Rio de Janeiro'
state='RJ'
country='BR' #dados da cidade que queremos registrar na nossa API

In [5]:
url='http://apiadvisor.climatempo.com.br/api/v1/locale/city?name={}&state={}&country={}&token={}'.format(city,state,country,token)
response=requests.get(url)
data=response.json()

print(data[1])
city_id=data[1]['id'] #pesquisando o id da cidade desejada

{'id': 73779, 'name': 'Rio de Janeiro - centro', 'state': 'RJ', 'country': 'BR  '}


In [6]:
url='http://apiadvisor.climatempo.com.br/api-manager/user-token/{}/locales'.format(token)

payload='localeId[]={}'.format(city_id)
headers={'Content-Type':'application/x-www-form-urlencoded'}

response=requests.put(url,headers=headers,data=payload)
response.json() #registrando a cidade no nosso app

{'status': 'success', 'locales': [73779]}

In [7]:
url='http://apiadvisor.climatempo.com.br/api/v1/forecast/locale/{}/days/15?token={}'.format(city_id,token)

response=requests.get(url)
data=response.json()

data.keys() #pegando a previsão para os próximos 7 dias na nossa cidade

dict_keys(['id', 'name', 'state', 'country', 'meteogram', 'data'])

In [8]:
city,state,country=data['name'],data['state'],data['country'] #salvando informações da cidade

In [9]:
week=data['data']
current_day=week[0]['date']
row=[current_day]

for day in week:
    row.append(str(day['rain']['probability'])+'%')
    
row.append('None')

In [10]:
today=datetime.strptime(current_day,'%Y-%m-%d').date()
dt=timedelta(days=1)
yesterday=str(today-dt)

In [11]:
df=pd.read_csv('tables/rain_probability_{}.csv'.format(yesterday))

df.loc[-1]=row
df.sort_values('date',ascending=False,inplace=True)
df.reset_index(drop=True,inplace=True)

In [12]:
df.to_csv('tables/rain_probability_{}.csv'.format(current_day),index=False)
df

,date,rain_probability_in_0_days,rain_probability_in_1_days,rain_probability_in_2_days,rain_probability_in_3_days,rain_probability_in_4_days,rain_probability_in_5_days,rain_probability_in_6_days,did_it_rain
0,2023-05-22,0%,0%,3%,0%,0%,2%,13%,None
1,2023-05-21,0%,0%,0%,3%,1%,1%,2%,None
2,2023-05-20,0%,0%,0%,2%,5%,4%,3%,0
3,2023-05-19,0%,0%,0%,0%,2%,5%,7%,0
4,2023-05-18,14%,1%,0%,0%,0%,4%,5%,0
5,2023-05-17,26%,14%,3%,1%,0%,0%,3%,1


In [15]:
#did_it_rain=0

In [16]:
df.iloc[1][-1]=did_it_rain
df.to_csv('tables/rain_probability_{}.csv'.format(current_day),index=False)
df

,date,rain_probability_in_0_days,rain_probability_in_1_days,rain_probability_in_2_days,rain_probability_in_3_days,rain_probability_in_4_days,rain_probability_in_5_days,rain_probability_in_6_days,did_it_rain
0,2023-05-22,0%,0%,3%,0%,0%,2%,13%,None
1,2023-05-21,0%,0%,0%,3%,1%,1%,2%,0
2,2023-05-20,0%,0%,0%,2%,5%,4%,3%,0
3,2023-05-19,0%,0%,0%,0%,2%,5%,7%,0
4,2023-05-18,14%,1%,0%,0%,0%,4%,5%,0
5,2023-05-17,26%,14%,3%,1%,0%,0%,3%,1
